# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load CSV file
cities_df = pd.read_csv('../WeatherPy/WeatherPy_Output.csv')

#Print
cities_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.6500,115.3333,62.01,83,0,1.01,AU,1612045501
1,1,Sakaiminato,35.5500,133.2333,37.40,93,40,6.91,JP,1612045502
2,2,Palencia,42.4167,-4.5000,39.99,89,27,7.00,ES,1612045499
3,3,Punta Arenas,-53.1500,-70.9167,55.40,58,0,21.85,CL,1612045326
4,4,Chui,-33.6971,-53.4616,75.52,71,59,14.00,UY,1612045506
...,...,...,...,...,...,...,...,...,...,...
567,567,Malbork,54.0359,19.0266,26.60,74,75,11.50,PL,1612046293
568,568,Jining,35.4050,116.5814,36.82,70,20,5.06,CN,1612046295
569,569,Taoudenni,22.6783,-3.9836,69.42,23,0,10.51,ML,1612046303
570,570,Mandalay,21.9747,96.0836,61.54,48,5,1.79,MM,1612046250


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations and humidity as the weights
heat_locations = cities_df[["Latitude", "Longitude"]]

humidity = cities_df["Humidity"]

figure_layout = {
    'width': '700px',
    'height': '400px'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='400px', width='700px'))

In [4]:
#Customize the layout
heat_layer.max_intensity=100
heat_layer.point_radius=3
heat_layer.dissipating=False

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the vacation spot conditions
temp_filtered_df = cities_df.loc[(cities_df['Max Temperature']) > 80, :]
vacation_df = temp_filtered_df.loc[(temp_filtered_df['Cloudiness']) <= 0, :]

#Print
vacation_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,Puerto Ayora,-0.7393,-90.3518,84.20,70,0,12.66,EC,1612045565
86,86,Santa Isabel,-23.3156,-46.2214,84.20,74,0,6.91,BR,1612045610
136,136,Geraldton,-28.7667,114.6000,82.40,30,0,12.66,AU,1612045673
155,155,Vila Velha,-20.3297,-40.2925,80.60,74,0,13.80,BR,1612045697
297,297,Touros,-5.1989,-35.4608,80.60,78,0,8.05,BR,1612045889
397,397,Minna,9.6139,6.5569,84.20,16,0,8.05,NG,1612046016
478,478,Kapoeta,4.7717,33.5903,81.68,31,0,10.07,SS,1612046161


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_df[['City', 'Country', 'Latitude', 'Longitude']]

hotel_df["Hotel Name"] = " "

hotel_df

<ipython-input-6-98073b8c11b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = " "


,City,Country,Latitude,Longitude,Hotel Name
50,Puerto Ayora,EC,-0.7393,-90.3518,
86,Santa Isabel,BR,-23.3156,-46.2214,
136,Geraldton,AU,-28.7667,114.6000,
155,Vila Velha,BR,-20.3297,-40.2925,
297,Touros,BR,-5.1989,-35.4608,
397,Minna,NG,9.6139,6.5569,
478,Kapoeta,SS,4.7717,33.5903,


In [7]:
# find the closest restaurant of each type to coordinates
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    city = row['City']
    
    # get location from df
    lat = row['Latitude']
    lng = row['Longitude']

    # add location to params dict
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #print(results)
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 50: Puerto Ayora.
Closest hotel is Castillo Galapagos.
------------
Retrieving Results for Index 86: Santa Isabel.


C:\Users\wazz_\Anaconda3.1\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Chácara Sombra do Altíssimo.
------------
Retrieving Results for Index 136: Geraldton.
Closest hotel is Broadwater Mariner Resort.
------------
Retrieving Results for Index 155: Vila Velha.
Closest hotel is Hotel Vitória Palace.
------------
Retrieving Results for Index 297: Touros.
Closest hotel is INN NEW HORIZON.
------------
Retrieving Results for Index 397: Minna.
Closest hotel is Royal Park hotel & suites.
------------
Retrieving Results for Index 478: Kapoeta.
Closest hotel is Eastgate Hotel.
------------


In [8]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
50,Puerto Ayora,EC,-0.7393,-90.3518,Castillo Galapagos
86,Santa Isabel,BR,-23.3156,-46.2214,Chácara Sombra do Altíssimo
136,Geraldton,AU,-28.7667,114.6000,Broadwater Mariner Resort
155,Vila Velha,BR,-20.3297,-40.2925,Hotel Vitória Palace
297,Touros,BR,-5.1989,-35.4608,INN NEW HORIZON
397,Minna,NG,9.6139,6.5569,Royal Park hotel & suites
478,Kapoeta,SS,4.7717,33.5903,Eastgate Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '700px',
    'height': '400px'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(heat_locations, weights=humidity)
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

#Customize the layout
heat_layer.max_intensity=100
heat_layer.point_radius=3
heat_layer.dissipating=False

# Display figure
fig

Figure(layout=FigureLayout(height='400px', width='700px'))